# CSC-321: Data Mining and Machine Learning
## Assignment 4: Multivariate Linear Regression


For more complex learning, we cannot simply read our coefficients (b0 and b1 for SLR) from the data. We have to approximate them because there are many more of them. If you remember, the function for SLR was:

y = b0 + b1*x

For the more complicated Multivariate Linear Regression the function is:

y = b0 + b1*x1 + b2*x2 + ... + bN*xN

With there being as many coefficients (b) as there are input features, plus 1 (for the b0 coefficient, or intercept). 

We don't know exactly how they play together for a given data set so we're going to use a mechanism to pick some random values, and gradually improve them over time.

The method we discussed in class is called Stochastic Gradient Descent, and is one of a variety of related optimization algorithms that are used in many machine learning methods to 'learn' the coefficients, or weights, on input values with respect to some output. 

Gradient descent it the process of minimizing some function (we'll call it the cost, or error function) by following the slope (or gradient) of the function down to some minimum. Intuitively, we're going to show our model one training instance at a time, make a prediction for that instance, calculate the error using that instance, and update the model in order to improve performance (get a smaller error) for the nect prediction. We'll repeat this process for a number of iterations, minimizing the error each time.

Each iteration, we're going to update the coefficients using the formula:

b = b - learning_rate * error * x

for each coefficient (again, corresponding to each input feature, of each instance).

Remember also that learning_rate is a value we must choose (I'll tell you to start with), and the number of iterations is ALSO a number we must choose. Let's start.

### Part 1: Making Predictions

(a) We're going to use predictions made by our model as a guide for tuning the coefficients, we need a function that can make predictions. This function can also apply the final coefficients we have learned to make predictions over new data. Write a function called predict(instance, coefficients), that takes a single instance (a list of input features), and a list of coefficients, and calculates the predicted y value, using the formula:

y = b0 + b1*x1 + b2*x2 + ... + bN*xN

This should work for ANY length of instance, but we can always assume that the length of the instance list and the length of the coefficent list are the SAME. For the instance, the values are:

[x1,x2,x3,...,xN,Y] (where Y is the actual value, in the case of training, or None, in the case of testing).

For the coefficients, let's assume that the list contains all the coefficients, including b0. Let's also assume that coefficients[0] is ALWAYS where we store b0. 

Your function predict(instance, coefficients) should return the predicted y value for a given instance and set of coefficients. 

(b) In the Simple Linear Regression assignment, you applied your model to a contrived data set. I've reproduced this below. Go through this data set one instance at a time and call your new predict function for each instance. You can use the coefficients [0.4, 0.8], which are almost exactly what you learned as coefficients in Assignment 2. 

For each instance, print out the correct value, and the value predicted by your function from (a). You should see that it performs reasonably well. 

In [1]:
# Write your predict function here
def predict(instance, coefficients):
    y = coefficients[0]
    for i in range(len(instance)-1):
        y += instance[i]*coefficients[i+1]
    return y

# Apply your function to the contrived dataset

dataset = [[1,1],[2,3],[4,3],[3,2],[5,5]]
coef = [0.4,0.8]

for i in range(len(dataset)):
    print('prediction:', predict(dataset[i], coef), 'real:', dataset[i][1])
    



prediction: 1.2000000000000002 real: 1
prediction: 2.0 real: 3
prediction: 3.6 real: 3
prediction: 2.8000000000000003 real: 2
prediction: 4.4 real: 5


# Part 2: Learning coefficients with Stochastic Gradient Descent

We now need to estimate coefficients for ourselves. For a given learning rate, for a number of iterations (epochs), we're going to estimate our coefficients.  Given a set of training data, we're going to:

- loop over each epoch
- loop over each row (instance) in the training data, for an epoch
- loop over each coefficient, for each feature in each instanc, for each epoch

As computer scientists you should recognize that this requires three, nested for loops, and you should have a sense (a big-O kind of sense) why this can take a long time for large data sets.

Coefficients are updated based on the error the model made. The error is calculated as the difference between the predicted y value and the actual y value:

error = prediction - actual

There is one coefficient for EACH input attribute, and these are updated every time, for example:

b1 = b1 - learning_rate * error * x1

We ALSO need to update the special intercept coefficient b0:

b0 = b0 - learning_rate * error

(c) Implement the following algorithm for Stochastic Gradient Descent, naming your function coefficientsSGD(train, learning_rate, epochs), where train is the training data, and the other two parameters are the ones that control the learning.

The algorithm is as follows: 

- initialize a list for the output coefficients. The length of the list will be the same as the length of every instance in the training data. We can initialize all the coefficients to 0.0 in the first instance
- for each epoch
    - initialize the total error to 0
    - for each instance in the training data
        - calculate the prediction for that instance, given the current list of coefficients, using our function from (a)
        - calculate the error for that prediction 
        - (remember, each instance of the training data has the actual Y value)
        - square the error, and add it to the total error. We're going to print the total error each time, and squaring the individual error means it will always be a positive value. NOTE: We don't use this squared error for updating the coefficients - we use the original error. This squaring is just to give us nice, readable output. 
        - Now update the coefficients, using the formulas given above. One update for b0 (which should always be at position 0 in the coefficients list), and then a series of updates for the remaining coefficients, b1 through bN.
        
    - At the end of each epoch, print out the epoch number (we can start at epoch 0), the learning rate, and the total error for this epoch.
- once we've iterated through each epoch, return the list of coefficients

(d) Apply your coefficientsSGD function to the contrived dataset, given below. If it's working, you should see the error rate falling each epoch. You should also note that the value of the coefficients learned isn't quite the same as Simple Linear Regression, because we're estimating each time. You could try learning longer (more epochs), or altering the learning rate, and see if the coefficients approach the optimal values we learned in Assignment 2. 

In [2]:
# Write your coefficientsSGD(train,learning_rate,epochs) here

def coefficientsSGD(train, learning_rate, epochs):
    coefficients = [0 for i in range(len(train[0]))]
    for e in range(epochs):
        totalError = 0
        for instance in train:
            predY = predict(instance, coefficients)
            error = predY - instance[-1]
            totalError += error**2
            coefficients[0] -= learning_rate*error
            for i in range(1,len(coefficients)):
                coefficients[i] -= learning_rate*error*instance[i-1]
        print('epoch=', e, 'lrate=', learning_rate, 'error=', totalError)
    return coefficients
                
# Apply to the contrived data here. Try my parameters first, before you experiment

dataset = [[1,1],[2,3],[4,3],[3,2],[5,5]]

learning_rate = 0.001
epochs = 50
coefs = coefficientsSGD(dataset, learning_rate, epochs)
print('Coefficients are:',coefs)


epoch= 0 lrate= 0.001 error= 46.23569225016471
epoch= 1 lrate= 0.001 error= 41.305142323835085
epoch= 2 lrate= 0.001 error= 36.92968879875065
epoch= 3 lrate= 0.001 error= 33.046843407651664
epoch= 4 lrate= 0.001 error= 29.601151923716085
epoch= 5 lrate= 0.001 error= 26.543402390484545
epoch= 6 lrate= 0.001 error= 23.82992247422831
epoch= 7 lrate= 0.001 error= 21.421955907121237
epoch= 8 lrate= 0.001 error= 19.285109118735654
epoch= 9 lrate= 0.001 error= 17.38886015544335
epoch= 10 lrate= 0.001 error= 15.706122876572774
epoch= 11 lrate= 0.001 error= 14.212860205347214
epoch= 12 lrate= 0.001 error= 12.88774091297372
epoch= 13 lrate= 0.001 error= 11.7118350357667
epoch= 14 lrate= 0.001 error= 10.668343576747102
epoch= 15 lrate= 0.001 error= 9.742358632631682
epoch= 16 lrate= 0.001 error= 8.920650521505976
epoch= 17 lrate= 0.001 error= 8.191478871959525
epoch= 18 lrate= 0.001 error= 7.544424976557279
epoch= 19 lrate= 0.001 error= 6.970243016110007
epoch= 20 lrate= 0.001 error= 6.4607280306

(e) Now you have sufficient functionality to write a function to make predictions using multivariate linear regression. Create a function with the signature mlr(train,test,learning_rate,epochs). Remember, training data is data containing the features of the data AND the class. Testing data contains the features, but does NOT contain a class (instead it should hold the value None in place of the class entry). 

We're going to use the same dataset here for both training and testing, even though we know that might not be a great idea.

Here's the mlr algorithm. We're going to estimate our coefficients from the training data, using the function from (c) above. We're going to create a new list, to hold our predictions. Then for each entry in the testing data, we're going to read the input value, and make a prediction, using our function from (a). For each entry in the test data, we're going to append our predicted y value to the prediction list. We're going to return our list of predictions.

In [3]:
# Write function mlr(train,test,learning_rate,epochs) here

def mlr(train, test, learning_rate, epochs):
    coefficients = coefficientsSGD(train, learning_rate, epochs)
    predictions = []
    for entry in test:
        prediction = predict(entry, coefficients)
        predictions.append(prediction)
    return predictions



### Part 3: Applying to real data

Last time I gave you the wine quality data set. Let's use that for this assignment. You'll need to:

(f) Load the data set
(g) Convert the features from strings to floats
(h) Normalize all the attributes
(i) Call the evaluate_algorithm function, given below with both your mlr algorithm, and a baseline algorithm (zeroRR). Print out the RMSE for both. 

Executing the above will require you to copy across a number of functions from the previous assignments.
At the end, write something about the result. To do that, it will be helpful to interpret both the final RMSE from the multivariate linear regression, and to know something about the original features. The wine data set is just the white wine component of the data you can find here: 

http://archive.ics.uci.edu/ml/datasets/Wine+Quality



In [4]:
# Write your code for f through i here
import csv

# Load the data
def load_data(filename):
    csvTxt = csv.reader(open(filename))
    data = []
    for row in csvTxt:
        data.append(row)
    return data

def column2Float(dataset,column):
    for instance in dataset:
        instance[column] = float(instance[column])
    return dataset

import math
def mean(listOfValues):
    total = 0
    for num in listOfValues:
        total += num
    return total/len(listOfValues)

def zeroRR(train, test):
    trainY = [i[-1] for i in train]
    testY = [i[-1] for i in test]

    trainYMean = mean(trainY)
    predictions = []
    for i in testY:
        predictions.append(trainYMean)
    return predictions


def rmse_eval(actual, predicted):
    error = 0.0
    for i in range(len(actual)):
        error += (predicted[i] - actual[i])**2
    error = error/len(actual)
    error = error**0.5
    return error

def minmax(dataset):
    listMinMax = []
    for column in range(len(dataset[0])):
        columnData = [dataset[i][column] for i in range(len(dataset))]
        listMinMax.append([min(columnData), max(columnData)])
    return listMinMax

def normalize(dataset, minmax):
    for row in range(len(dataset)):
        for column in range(len(dataset[row])):
            dataset[row][column] = (dataset[row][column] - minmax[column][0]) / (minmax[column][1] - minmax[column][0])

wineData = load_data('winequality-white.csv')
#print('loaded wine data', wineData[0])
for column in range(len(wineData[0])):
    column2Float(wineData, column)
#print('wine data as floats', wineData[0])
minmaxWine = minmax(wineData)
normalize(wineData, minmaxWine)
#print('wine data normalized', wineData[0])

def evaluate_algorithm(dataset, algorithm, metric, *args):
    train = dataset
    test = [dataset[i][:-1] for i in range(len(dataset))]
    for i in test:
        i.append(None)
    predicted = algorithm(train,test,*args)
    actual = [i[-1] for i in dataset]
    result = metric(actual,predicted)
    return result


# Testing multivariate linear regression

dataset = wineData
learning_rate = 0.01
epochs = 50
mlr_result = evaluate_algorithm(dataset,mlr,rmse_eval,learning_rate,epochs)
zeroRR_result = evaluate_algorithm(dataset,zeroRR,rmse_eval)

print('MLR RMSE: %.3f' % mlr_result)
print('zeroRR RMSE: %.3f' % zeroRR_result)


epoch= 0 lrate= 0.01 error= 94.49427503765666
epoch= 1 lrate= 0.01 error= 80.86482419578945
epoch= 2 lrate= 0.01 error= 78.74976548229141
epoch= 3 lrate= 0.01 error= 77.65986226752669
epoch= 4 lrate= 0.01 error= 77.03161826055162
epoch= 5 lrate= 0.01 error= 76.64462946945235
epoch= 6 lrate= 0.01 error= 76.39423515123764
epoch= 7 lrate= 0.01 error= 76.22594572954972
epoch= 8 lrate= 0.01 error= 76.10930852419794
epoch= 9 lrate= 0.01 error= 76.02627901751154
epoch= 10 lrate= 0.01 error= 75.96565474337895
epoch= 11 lrate= 0.01 error= 75.92023083082891
epoch= 12 lrate= 0.01 error= 75.88525248856908
epoch= 13 lrate= 0.01 error= 75.85752342887567
epoch= 14 lrate= 0.01 error= 75.8348665047151
epoch= 15 lrate= 0.01 error= 75.81578448152017
epoch= 16 lrate= 0.01 error= 75.79924030111283
epoch= 17 lrate= 0.01 error= 75.78451162948105
epoch= 18 lrate= 0.01 error= 75.77109304880543
epoch= 19 lrate= 0.01 error= 75.75862952887735
epoch= 20 lrate= 0.01 error= 75.74687078919008
epoch= 21 lrate= 0.01 er

#### Part 2 Discussion Here

The MLR found wine quality 0.03 closer to the actual wine quality than zeroRR using RMSE as the evaluation metric. The wine quality is on a scale of 0 to 10 so an improvement of 0.03 does not seem that great for the effort involved. I briefly looked over the data and most of the wine quality was either 5, 6, or 7 which would be around the mean. This makes sense as to why the zeroR worked so well because most of the ouput data was near the mean.
